In [2]:
import pandas as pd 
import seaborn as sns
sns.set(font_scale=1)
from plotnine import *
from tqdm.notebook import tqdm
import seaborn as sns
import pandas as pd
from pandarallel import pandarallel

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)
tqdm.pandas()

In [3]:
PAIRED_DATASETS = pd.read_csv("/home/pbarbosa/git_repos/mutsplice/data/2_paired_datasets/ALL_data.tsv.gz", sep="\t")

In [4]:
from gtfhandle.utils import file_to_bed_df
from mutsplice.datasets.mutsplice_pipeline import MutSplicePipeline
import os

def runMutSplice(df, rbp_name: str, exon_group: str, motif_source: str, motif_search: str):
    GTF_CACHE = "/home/pbarbosa/data/genomes/hg38/gtf_cache_gencode/"
    OUT_DIR = f"/big_data/pbarbosa/mutsplice/{motif_source}_{motif_search}/"
    FASTA = "/home/pbarbosa/data/genomes/hg38/GRCh38.primary_assembly.genome.fa"

    kwargs = {
        "gtf_cache": GTF_CACHE,
        "fasta": FASTA,
        "out_dir": f"{OUT_DIR}{rbp_name}_{exon_group}",
        "outbasename": f"{rbp_name}",
        "subset_rbps": [rbp_name],
        "motif_source": motif_source,
        "motif_search": motif_search,
        "pvalue_threshold": 0.0005,
        "spliceai_save_raw": True,
        "spliceai_raw_results": None,
        "spliceai_final_results": None,
        "batch_size": 64,
        "use_full_sequence": False,
    }
    bed_df = file_to_bed_df(df, is_0_based=False, header=0, col_index=0)

    MutSplicePipeline(
        bed_df,
        do_gtf_queries=True,
        do_motif_scanning=True,
        do_mutations=True,
        run_spliceai=True,
        **kwargs,
    )


def process_single_RBP(group, exon_group: str, motif_source: str, motif_search: str):
    try:
        rbp_name = group.iloc[0].rbp_name
        runMutSplice(
            group,
            rbp_name=rbp_name,
            exon_group=exon_group,
            motif_source=motif_source,
            motif_search=motif_search,
        )

    except ValueError as ve:
        print(f"Caught ValueError for {rbp_name}: {ve}")

2024-03-26 15:40:59.019448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 15:41:00.224933: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-26 15:41:00.225014: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-26 15:41:00.225021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

#### Knockdown exons

In [7]:
KD_data

,target_coordinates,Strand,gene_name,transcript_id,rbp_name,exon_group,dPSI,paired_with
0,chr2:130161186-130161272,-,SMPD4,ENST00000680298,ADAR,KD,0.229,chr22:35338713-35338788
2,chr11:47259180-47259259,+,NR1H3,ENST00000441012,ADAR,KD,0.442,chr20:37401602-37401678
4,chr20:49850810-49850844,+,SLC9A8,ENST00000361573,ADAR,KD,0.612,chr15:84679932-84679969
6,chr9:96852853-96853069,-,ZNF782,ENST00000481138,ADAR,KD,0.576,chr15:72349079-72349259
8,chr9:133465322-133465447,+,CACFD1,ENST00000316948,ADAR,KD,-0.273,chr7:857828-857957
...,...,...,...,...,...,...,...,...
30460,chr14:91145221-91145313,+,DGLUCY,ENST00000518868,ZRANB2,KD,-0.598,chr16:28581055-28581144
30462,chr12:114679913-114679972,-,TBX3,ENST00000257566,ZRANB2,KD,-0.174,chr11:62571838-62571901
30464,chr10:96954953-96955834,+,LCOR,ENST00000540664,ZRANB2,KD,0.419,chr2:61214060-61214694
30466,chr10:96954953-96955671,+,LCOR,ENST00000676381,ZRANB2,KD,0.411,chr2:23757418-23758101


In [6]:
pandarallel.initialize(progress_bar=True, nb_workers=1)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

KD_data = PAIRED_DATASETS[PAIRED_DATASETS.exon_group == "KD"]
# KD_data.groupby("rbp_name").parallel_apply(
#     process_single_RBP, exon_group="KD", motif_source="ATtRACT", motif_search="fimo"
# )

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


#### Control exons 

In [ ]:
pandarallel.initialize(progress_bar=True, nb_workers=1)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Ctrl_data = PAIRED_DATASETS[PAIRED_DATASETS.exon_group == "CTRL"]
Ctrl_data.groupby("rbp_name").parallel_apply(
    process_single_RBP,
    exon_group="CTRL",
    motif_source="ATtRACT",
    motif_search="fimo",
)